<a href="https://colab.research.google.com/github/Kkuntal990/CTW_2020/blob/master/AE_trials.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np # linear algebra
import tensorflow as tf
import numpy as np
import os
import h5py
import pandas as pd
import time
import matplotlib.pyplot as plt


In [0]:
from keras.models import Sequential, Model, load_model
from keras.layers import Dense, LeakyReLU, Input, Conv2D, AveragePooling2D, MaxPooling2D, Dropout, Flatten
from keras.regularizers import l1,l2

In [42]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [0]:

def load_unlabelled2(path):
    f = h5py.File(path, 'r')
    '''
    H_Re = f['H_Re'][:] #shape (sample size, 56, 924, 5)
    H_Im = f['H_Im'][:] #shape (sample size, 56, 924, 5)
    SNR  = f['SNR'][:]  #shape (sample size, 56, 5)
    print(SNR.shape)
    '''
  
    print(f.keys())
    H = f['h_Estimated'][:]
    f.close()
    return H


def load_unlabelled3(path):
    f = h5py.File(path, 'r')
    '''
    H_Re = f['H_Re'][:] #shape (sample size, 56, 924, 5)
    H_Im = f['H_Im'][:] #shape (sample size, 56, 924, 5)
    SNR  = f['SNR'][:]  #shape (sample size, 56, 5)
    print(SNR.shape)
    '''
  
    print(f.keys())
    H = f['r_Position'][:]
    f.close()
    return H


def load_unlabelled(path):
    f = h5py.File(path, 'r')
    '''
    H_Re = f['H_Re'][:] #shape (sample size, 56, 924, 5)
    H_Im = f['H_Im'][:] #shape (sample size, 56, 924, 5)
    SNR  = f['SNR'][:]  #shape (sample size, 56, 5)
    print(SNR.shape)
    '''
  
    print(f.keys())
    H = f['SNR_Est'][:]
    f.close()
    return H

In [0]:
H = load_unlabelled2('/content/drive/My Drive/CTW_2019/h_Estimated_CTW_Train.h5')
pos = load_unlabelled3('/content/drive/My Drive/CTW_2019/r_Position_CTW_Train.h5')

In [0]:
SNR = load_unlabelled('/content/drive/My Drive/CTW_2019/SNR_CTW_Train.h5')

In [0]:
print(H.shape)
print(pos.shape)
print(SNR.shape)
Y = np.swapaxes(pos, 0,1)
print(Y.shape)
H1 = np.swapaxes(H,0,3)
print(H1.shape)
print(H[:,1:10,1,1])
print(H1[1,1:10,1,:])

In [0]:
def create_model():
    inp = Input(shape=(2,16,924))
    x = Dense(256, activation='relu',activity_regularizer=l1(10e-4))(inp)
  #  x = Dense(128, activation='relu')(x)
    encoded = Dense(64, activation='relu',activity_regularizer=l1(10e-4))(x)
  #  x = Dense(128, activation='relu')(encoded)
    x = Dense(256, activation='relu',activity_regularizer=l1(10e-4))(encoded)
    decoded = Dense(924, activation='relu')(x)
    return Model(inp, encoded), Model(inp, decoded)


In [0]:
from sklearn.model_selection import train_test_split
X_train,X_test = train_test_split(H1, test_size=0.1, random_state=42, shuffle=True)
y_train,y_test = train_test_split(Y, test_size=0.1, random_state=42, shuffle=True)

print(X_train.shape,X_test.shape)
print(y_train.shape,y_test.shape)


In [0]:
X_train = np.swapaxes(X_train,1,3)
print(X_train.shape)
print(H1[1,1:10,1,:])
print(np.swapaxes(H1,1,3)[1,:,1,1:10])

In [38]:
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.optimizers import Adadelta, Adam, RMSprop


H_comp, H_decod = create_model()

H_decod.compile(optimizer='Adam', loss = 'mean_squared_error')
early_stopping = EarlyStopping(monitor='val_loss', patience = 20)
checkpointer = ModelCheckpoint('/content/drive/My Drive/CTW_2019/SimpleAE.h5', verbose=1, save_best_only=True)
H_decod.summary()

Model: "model_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_8 (InputLayer)         (None, 2, 16, 924)        0         
_________________________________________________________________
dense_17 (Dense)             (None, 2, 16, 256)        236800    
_________________________________________________________________
dense_18 (Dense)             (None, 2, 16, 64)         16448     
_________________________________________________________________
dense_19 (Dense)             (None, 2, 16, 256)        16640     
_________________________________________________________________
dense_20 (Dense)             (None, 2, 16, 924)        237468    
Total params: 507,356
Trainable params: 507,356
Non-trainable params: 0
_________________________________________________________________


In [39]:
H_decod.fit(X_train, X_train, epochs = 150, verbose=1, batch_size = 32,validation_split=0.1, callbacks = [checkpointer,early_stopping])

Train on 14163 samples, validate on 1574 samples
Epoch 1/150
14163/14163 [==============================] - 6s 454us/step - loss: 0.0704 - val_loss: 0.0040

Epoch 00001: val_loss improved from inf to 0.00405, saving model to /content/drive/My Drive/CTW_2019/SimpleAE.h5
Epoch 2/150
14163/14163 [==============================] - 6s 428us/step - loss: 0.0036 - val_loss: 0.0035

Epoch 00002: val_loss improved from 0.00405 to 0.00346, saving model to /content/drive/My Drive/CTW_2019/SimpleAE.h5
Epoch 3/150
14163/14163 [==============================] - 6s 439us/step - loss: 0.0031 - val_loss: 0.0029

Epoch 00003: val_loss improved from 0.00346 to 0.00294, saving model to /content/drive/My Drive/CTW_2019/SimpleAE.h5
Epoch 4/150
14163/14163 [==============================] - 6s 436us/step - loss: 0.0027 - val_loss: 0.0026

Epoch 00004: val_loss improved from 0.00294 to 0.00262, saving model to /content/drive/My Drive/CTW_2019/SimpleAE.h5
Epoch 5/150
14163/14163 [==============================